In [6]:
import os
import shutil
from pathlib import Path
from ultralytics import YOLO

In [ ]:
# Thiết lập đường dẫn
root_directory = "C:/Model/Fruits-detection"
target_directory = "C:/Model/organized_dataset"

Path(target_directory).mkdir(parents=True, exist_ok=True)

subdirs = ['test', 'train', 'valid']
image_extensions = ['.jpg', '.jpeg', '.png']
processed_images = set()
class_ids = set()

for subdir in subdirs:
    source_dir = Path(root_directory) / subdir
    if not source_dir.exists():
        print(f"Thư mục {subdir} không tồn tại")
        continue
    
    print(f"Đang xử lý thư mục: {subdir}")
    
    images_dir = source_dir / 'images'
    labels_dir = source_dir / 'labels'
    
    if not images_dir.exists() or not labels_dir.exists():
        print(f"Không tìm thấy images hoặc labels trong {subdir}")
        continue
    
    print(f"Danh sách file ảnh: {os.listdir(images_dir)}")
    print(f"Danh sách file label: {os.listdir(labels_dir)}")
    
    for label_file in labels_dir.glob('*.txt'):
        image_name = label_file.stem
        
        image_path = None
        for ext in image_extensions:
            potential_image = images_dir / f"{image_name}{ext}"
            if potential_image.exists():
                image_path = potential_image
                break
                
        if not image_path:
            print(f"Không tìm thấy file ảnh cho {label_file} trong {subdir}")
            continue
            
        with open(label_file, 'r') as f:
            first_line = f.readline().strip()
            if not first_line:
                print(f"File label {label_file} trống rỗng")
                continue
            class_id = first_line.split()[0]
            class_ids.add(class_id)
            
        class_dir = Path(target_directory) / subdir / f"class_{class_id}"
        class_dir.mkdir(parents=True, exist_ok=True)
        
        target_images_dir = class_dir / "images"
        target_labels_dir = class_dir / "labels"
        target_images_dir.mkdir(parents=True, exist_ok=True)
        target_labels_dir.mkdir(parents=True, exist_ok=True)
        
        new_image_path = target_images_dir / image_path.name
        new_label_path = target_labels_dir / label_file.name
        
        shutil.copy2(image_path, new_image_path)
        shutil.copy2(label_file, new_label_path)
        
        processed_images.add(image_name)
        print(f"Đã xử lý: {image_name} trong {subdir}")

print(f"\nĐã xử lý tổng cộng {len(processed_images)} ảnh và label")


for subdir in subdirs:
    source_images_dir = Path(root_directory) / subdir / 'images'
    if source_images_dir.exists():
        for ext in image_extensions:
            for image_file in source_images_dir.glob(f"*{ext}"):
                if image_file.stem not in processed_images:
                    print(f"Cảnh báo: Ảnh {image_file.name} trong {subdir} không có file label tương ứng")

Đang xử lý thư mục: test
Danh sách file ảnh: ['000aee0af66d4237_jpg.rf.6b99256606e5b7a1ede8b833a87196a7.jpg', '000d9c59687b509b_jpg.rf.c338360b3dfe1953a5d266ee861d7e79.jpg', '000e4e7ed48c932d_jpg.rf.8233751e16b32b9470e45c6135c8f8c7.jpg', '00149fd0a01cc623_jpg.rf.68e2b35f1bc31938c4a10d32a8f5340e.jpg', '006e0a3d99d16d60_jpg.rf.258f5af33beab4d59918218c732de430.jpg', '0074691e66aa85c0_jpg.rf.2fc1e81bbe5377b89f9c912c52ff7905.jpg', '009b02d58c46ed73_jpg.rf.923b83d2b5656ae821e1e32781393aa2.jpg', '00a5ae275e1ebe39_jpg.rf.be4ae1c307aaeaa6d46752808eda2ad8.jpg', '00a70c466ad1fe24_jpg.rf.cf298c47a082719e705775c1ec343775.jpg', '00b7cee6cbe6af50_jpg.rf.3a3cae04aadca322e29eafacce1af649.jpg', '00d843af60eecf7c_jpg.rf.868bfcbb7f5f2cc63f82441d7ddcf577.jpg', '01136d0450db721e_jpg.rf.40fd9b3238a46e4275c749797ec4c2be.jpg', '0113f90219ef40da_jpg.rf.b0c40161be08c4532f372d07a968e2ee.jpg', '013f0ff1e9186bea_jpg.rf.819554b9dbd9eeaea8928ec4619c1ce0.jpg', '01918ed451cb9a4f_jpg.rf.a6773a8b304b6909fb5cf73e6390bc44.

In [ ]:
# Thiết lập đường dẫn
target_directory = "C:/Model/organized_dataset"
data_yaml_path = "C:/Model/data.yaml"

# Tạo nội dung file data.yaml
nc = len(class_ids)
names = ["apple", "banana", "grape", "orange", "pineapple", "watermelon"]

data_yaml = f"""train: {Path(target_directory) / 'train'}
val: {Path(target_directory) / 'valid'}
test: {Path(target_directory) / 'test'}

nc: {nc}  # Số lượng class
names: {names}  # Tên các class (có thể thay bằng tên thực tế)
"""

with open(data_yaml_path, 'w', encoding='utf-8') as f:
    f.write(data_yaml)
print(f"Đã tạo file data.yaml tại: {data_yaml_path}")

Đã tạo file data.yaml tại: C:/Model/data.yaml


In [ ]:
# Thiết lập đường dẫn
data_yaml_path = "C:/Model/data.yaml"

# Khởi tạo mô hình
model = YOLO("yolov8n.pt")

# Cấu hình huấn luyện
epochs = 50
imgsz = 640
batch_size = 16

# Huấn luyện
results = model.train(
    data=data_yaml_path,
    epochs=epochs,
    imgsz=imgsz,
    batch=batch_size,
    project="C:/Model/runs",
    name="train_fruits_final",
    exist_ok=True,
    device=0
)

print("Huấn luyện hoàn tất!")
print(f"Kết quả được lưu tại: C:/Model/runs/train_fruits")

New https://pypi.org/project/ultralytics/8.3.94 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.91  Python-3.11.9 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:/Model/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=C:/Model/runs, name=train_fruits_final, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embe

train: Scanning C:\Model\organized_dataset\train\class_0\labels.cache... 7108 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7108/7108 [00:00<?, ?it/s]

train: WARNING  C:\Model\organized_dataset\train\class_5\images\3d8be4f881b8c54c_jpg.rf.0d7b6d095459cece040b47b246d807af.jpg: 1 duplicate labels removed
train: WARNING  C:\Model\organized_dataset\train\class_5\images\3d8be4f881b8c54c_jpg.rf.64e869a9bedd5f012cc2a1129c6ca229.jpg: 1 duplicate labels removed



val: Scanning C:\Model\organized_dataset\valid\class_0\labels.cache... 914 images, 0 backgrounds, 0 corrupt: 100%|██████████| 914/914 [00:00<?, ?it/s]

val: WARNING  C:\Model\organized_dataset\valid\class_1\images\3d3ddc3054b32eb7_jpg.rf.03e7789aaf5212e2634b84ef502e0832.jpg: 1 duplicate labels removed


...

      50/50      3.47G     0.6531     0.5599      1.017          9        640: 100%|██████████| 445/445 [01:28<00:00,  5.05it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:06<00:00,  4.42it/s]

                   all        914       3227       0.65      0.454        0.5      0.338



50 epochs completed in 1.298 hours.
Optimizer stripped from C:\Model\runs\train_fruits_final\weights\last.pt, 6.3MB
Optimizer stripped from C:\Model\runs\train_fruits_final\weights\best.pt, 6.3MB

Validating C:\Model\runs\train_fruits_final\weights\best.pt...
Ultralytics 8.3.91  Python-3.11.9 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:07<00:00,  4.05it/s]


                   all        914       3227      0.622      0.471      0.503      0.338
                 apple        188        557      0.644      0.483      0.515       0.36
                banana        167        390      0.613       0.49      0.519      0.323
                 grape        199        809      0.602      0.381        0.4      0.261
                orange        197       1100      0.617      0.383      0.432       0.28
             pineapple         77        154      0.675      0.506      0.556      0.362
            watermelon        107        217      0.583      0.585      0.595      0.439
Speed: 0.4ms preprocess, 3.2ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to C:\Model\runs\train_fruits_final
Huấn luyện hoàn tất!
Kết quả được lưu tại: C:/Model/runs/train_fruits
